In [2]:
!pip install pandas
!pip install openpyxl
!pip install fhir.resources

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
from IPython.display import display, HTML
import os
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.observation import Observation
from fhir.resources.R4B.observation import ObservationComponent
from fhir.resources.R4B.procedure import Procedure
from fhir.resources.R4B.procedure import ProcedurePerformer
from fhir.resources.R4B.condition import Condition
from fhir.resources.R4B.allergyintolerance import AllergyIntolerance
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.quantity import Quantity
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone, timedelta
from fhir.resources.R4B.questionnaireresponse import (
    QuestionnaireResponse,
    QuestionnaireResponseItem,
    QuestionnaireResponseItemAnswer
)


C:\Users\drtho\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
#Defining BRU_metadata for exporting into Bruno
BRU_metadata= """
meta {{
    name: {Filename}
    type: http
    seq: {Sequence}
}}

post {{
    url: {{{{base_url}}}}/{Resource_Type}
    body: json
    auth: bearer
}}

auth:bearer {{
    token: {{{{token}}}}
}}

body:json {{
{Result_JSON}
}}
"""

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function == dependency for to_JSON functions
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Operation based on observation input choices
ObservationCategory_Dict = {
    "Activity": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "activity",
        "display": "Activity"
    },
    "Exam": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "exam",
        "display": "Exam"
    },
    "Imaging": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "imaging",
        "display": "Imaging"
    },
    "Laboratory": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "laboratory",
        "display": "Laboratory"
    },
    "Procedure": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "procedure",
        "display": "Procedure"
    },
    "Social History": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "social-history",
        "display": "Social History"
    },
    "Survey": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "survey",
        "display": "Survey"
    },
    "Therapy": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "therapy",
        "display": "Therapy"
    },
    "Vital Signs": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "vital-signs",
        "display": "Vital Signs"
    }
}

#Definining Observation_IDFHIR Class
class Observation_IDFHIR:
    def __init__(self):
        self.observation = Observation(
            # Initialize necessary Observation attributes here
                status= Observation_Status,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Observation/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[observation_coding]),
                category = [CodeableConcept(coding=[category_coding])],
                subject = Subject,
                encounter = Encounter,
                effectiveDateTime= DateTime,
                issued= DateTime,
                performer= [Performer]
        )

    def set_value(self, value_type):
        if value_type == "valueQuantity":
            self.observation.valueQuantity = Quantity(
                value= Observation_valueQuantityValue, 
                code= Observation_valueQuantityCode, 
                system= Observation_valueQuantitySystem, 
                unit= Observation_valueQuantityUnit)
        #elif value_type == "valueDateTime":
        #    self.observation.valueDateTime = Observation_valueDateTime + "+00:00"
        elif value_type == "valueString":
            self.observation.valueString = Observation_valueString
        elif value_type == "valueCodeableConcept":
            valueCodeableConcept_coding= Coding(
                system= Observation_valueCodeSystem,
                code= Observation_valueCodeCode,
                display= Observation_valueCodeDisplay
            )
            self.observation.valueCodeableConcept = CodeableConcept(coding=[valueCodeableConcept_coding])
        elif value_type == "valueBoolean":
            self.observation.valueBoolean = Observation_valueBoolean
        elif value_type == "valueInteger":
            self.observation.valueInteger= Observation_valueInteger
        else:
            raise ValueError("Invalid value type")
        
    def set_interpretation (self, interpretation_type):
        if interpretation_type == "Used":
            interpretation_coding = Coding(
                system=ObservationInterpretation_System,
                code=ObservationInterpretation_Code,
                display=ObservationInterpretation_Display
            )
            self.observation.interpretation = [CodeableConcept(coding=[interpretation_coding])]
        else:
            self.observation.interpretation = None
    
    def to_dict(self):
        """Convert the Observation resource to a dictionary."""
        return self.observation.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Observation resource to a JSON string."""
        return self.observation.json(indent=4)# returns beautified JSON

class ObservationComponent_IDFHIR:
    def __init__(self):
        self.component= ObservationComponent(
            code= CodeableConcept(coding= [observation_coding])
        ) 
        return None

    def set_value(self, value_type):
        if value_type == "valueQuantity":
            self.component.valueQuantity = Quantity(
                value= Observation_valueQuantityValue, 
                code= Observation_valueQuantityCode, 
                system= Observation_valueQuantitySystem, 
                unit= Observation_valueQuantityUnit
            )
        #elif value_type == "valueDateTime":
        #    self.valueDateTime = Observation_valueDateTime + "+00:00"
        elif value_type == "valueString":
            self.component.valueString = Observation_valueString
        elif value_type == "valueCodeableConcept":
            valueCodeableConcept_coding= Coding(
                system= Observation_valueCodeSystem,
                code= Observation_valueCodeCode,
                display= Observation_valueCodeDisplay
            )
            self.component.valueCodeableConcept = CodeableConcept(coding=[valueCodeableConcept_coding])
        elif value_type == "valueBoolean":
            self.component.valueBoolean = Observation_valueBoolean
        elif value_type == "valueInteger":
            self.component.valueInteger= Observation_valueInteger
        else:
            raise ValueError("Invalid value type")
        pass

#Defining Resource_Construct Class
class Resource_Construct:
    def __init__(self, DataFrame, DateTime):
        for index, row in DataFrame.iterrows():
            MR_Gen= random_with_N_digits(4)
            Main_Folder = row['Main_Folder']
            Exam_Name = row['Folder_Name']
            JSON_Name = row['JSON_Title']
            Resource_Type= row["Resource_Type"]
    
            if Resource_Type == "Observation":
                identifier_system= b'http://sys-ids.kemkes.go.id/Observation/{{Org_id}}'
                identifier_value= '202403A000' + str(MR_Gen)
                Observation_Status= row["Status"]
                Observation_Category= row['Category']
                Observation_CodeCodingSystem = row['Code_system']
                Observation_CodeCodingCode = row['Code_Code']
                Observation_CodeCodingDisplay = row['Code_Display']
                Observation_valueType = row['Value_Type']
                Observation_valueInteger = row['valueInteger']
                Observation_valueCodeSystem = row['CodeableConcept_System']
                Observation_valueCodeCode = row['CodeableConcept_Code']
                Observation_valueCodeDisplay = row['CodeableConcept_Display']
                Observation_valueBoolean = row['valueBoolean']
                Observation_valueQuantityValue = row['valueQuantity_value']
                Observation_valueQuantityUnit = row['valueQuantity_unit']
                Observation_valueQuantitySystem = row['valueQuantity_System']
                Observation_valueQuantityCode = row['valueQuantity_code']
                Observation_valueString = row['valueString']
                ObservationInterpretation_Type= row['Interpretation_Used']
                ObservationInterpretation_System= row['Interpretation_System']
                ObservationInterpretation_Code= row['Interpretation_Code']
                ObservationInterpretation_Display= row ['Interpretation_Display']
                DateTime= DateTime + timedelta(seconds= 60)

                # Set up category
                category_coding = Coding(
                    system=ObservationCategory_Dict[Observation_Category]["system"], 
                    code=ObservationCategory_Dict[Observation_Category]["code"], 
                    display=ObservationCategory_Dict[Observation_Category]["code"]
                )

                # Set up code
                observation_coding = Coding(
                    system=Observation_CodeCodingSystem, 
                    code=Observation_CodeCodingCode, 
                    display=Observation_CodeCodingDisplay
                )

                resource= Observation_IDFHIR()
                resource= resource.set_value(Observation_valueType)
                resource= resource.set_interpretation(ObservationInterpretation_Type)
        
            resource= resource.json(indent= 4)      
            print(x," - ",Main_Folder,'\n',Exam_Name,'\n',JSON_Name)
            num_rows = resource.count('\n')
            html_text = f'''
            <textarea id="clipboard-text" rows="{num_rows}" cols="150">{resource}</textarea>
            <button onclick="copyToClipboard()">Copy text</button>
            <script>
            function copyToClipboard() {{
                var copyText = document.getElementById("clipboard-text");
                copyText.select();
                navigator.clipboard.writeText(copyText.value);
                }}
            </script>
            '''
            display(HTML(html_text))
            print('=======================================================','\n','\n','\n','\n')
            
            #Add indentation to every line in Result_JSON
            Result_JSON = '\n'.join('\t'+ line for line in resource.split('\n'))
            formatted_json = BRU_metadata.format(
                Filename= JSON_Name,
                Sequence= x,
                Result_JSON= Result_JSON
            )
            
            # Define the target folder and file name
            folder_path = f'{Main_Folder}\\{Exam_Name}'  # Update this path
            file_name = f'{JSON_Name}.bru'

            # Ensure the folder exists
            os.makedirs(folder_path, exist_ok=True)
            
            #Joining paths as full path and write the JSON inside that path
            full_path = os.path.join(folder_path, file_name)
            open(full_path, 'w').write(formatted_json)
            
            #Adding along sequence numbers for reuse
            x += 1
            MR_Gen += 1
    
    def to_dict(self):
        """Convert the Observation resource to a dictionary."""
        return self.observation.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Observation resource to a JSON string."""
        return self.observation.json(indent=4)# returns beautified JSON

#Definining Procedure_IDFHIR Class
class Procedure_IDFHIR:
    def __init__(self):
        self.Procedure = Procedure(
            # Initialize necessary Procedure attributes here
                status=Procedure_Status,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Procedure/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[Procedure_coding]),
                category = CodeableConcept(coding=[category_coding]),
                subject = Subject,
                encounter = Encounter,
                performedDateTime= DateTime,
                recorder= Reference(reference= Practitioner_Id, display= Practitioner_String),
                performer= [Procedure_Performer]
        )
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Procedure.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Procedure.json(indent=4)# returns beautified JSON

ConditionCategory_Dict= {
    "Problem List Item": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "problem-list-item",
        "display": "Problem List Item"
    },
    "Encounter Diagnosis": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "encounter-diagnosis",
        "display": "Encounter Diagnosis"
    }
}

ConditionClinicalStatus_Dict= {
    "active": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active",
        "display": "Active"
    },
    "recurrence": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "recurrence",
        "display": "Recurrence"
    },
    "relapse": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "relapse",
        "display": "Relapse"
    },
    "inactive": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "inactive",
        "display": "Inactive"
    },
    "remission": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "remission",
        "display": "Remission"
    },
    "resolved": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
    }
}

#Definining Condition_IDFHIR Class
class Condition_IDFHIR:
    def __init__(self):
        self.Condition = Condition(
            # Initialize necessary Procedure attributes here
                clinicalStatus= Condition_ClinicalStatus,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Condition/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[Condition_coding]),
                category = [CodeableConcept(coding=[category_coding])],
                subject = Subject,
                encounter = Encounter,
                recordedDate= DateTime,
                recorder= Reference(reference= Practitioner_Id, display= Practitioner_String),
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Condition.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Condition.json(indent=4)# returns beautified JSON

AllergyIntoleranceClinicalStatus_Dict= {
    "active": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "active",
        "display": "Active"
    },
    "inactive": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "inactive",
        "display": "Inactive"
    },
    "resolved": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "resolved",
        "display": "Resolved"
    }
}

#Definining Condition_IDFHIR Class
class AllergyIntolerance_IDFHIR:
    def __init__(self):
        self.AllergyIntolerance = AllergyIntolerance(
            # Initialize necessary Procedure attributes here
                clinicalStatus= CodeableConcept(coding= [AllergyIntolerance_ClinicalStatus_Coding]),
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Allergy/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[AllergyIntolerance_coding]),
                category = [AllergyIntolerance_Category],
                patient = Subject,
                encounter = Encounter,
                recordedDate= DateTime,
                recorder= Reference(reference= Practitioner_Id, display= Practitioner_String),
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.AllergyIntolerance.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.AllergyIntolerance.json(indent=4)# returns beautified JSON


In [5]:
file_xlsx = r"C:\Users\drtho\OneDrive\Documents\JSONs for Postman SATUSEHAT\Modul2 baru\Metadata SiGIZI - for automation.xlsx"
df_main= pd.read_excel(file_xlsx, dtype=str)
df_main

Sequence                 Main_Folder  \
0          1                      SiGIZI   
1          2                      SiGIZI   
2          3                      SiGIZI   
3          4                      SiGIZI   
4          5                      SiGIZI   
..       ...                         ...   
100       86  Gizi Buruk (Pelita Kesmas)   
101       87  Gizi Buruk (Pelita Kesmas)   
102       88  Gizi Buruk (Pelita Kesmas)   
103       89  Gizi Buruk (Pelita Kesmas)   
104       90  Gizi Buruk (Pelita Kesmas)   

                                    Folder_Name  \
0                                  Antropometri   
1                                  Antropometri   
2                                  Antropometri   
3                                  Antropometri   
4                                  Antropometri   
..                                          ...   
100  Penanganan yang diberikan untuk rawat inap   
101  Penanganan yang diberikan untuk rawat inap   
102  Penanganan yang diberikan untuk rawat inap   
103  Penanganan yang diberikan untuk rawat inap   
104  Penanganan yang diberikan untuk rawat inap   

                                            JSON_Title            Value_Type  \
0                      Observation - Berat Badan Lahir         valueQuantity   
1                            Observation - Berat Badan         valueQuantity   
2           Observation - Kenaikan Berat Badan Adekuat          valueBoolean   
3       Observation - Persentase Penurunan Berat Badan  valueCodeableConcept   
4                                   Observation - BB/U         valueQuantity   
..                                                 ...                   ...   
100  Observation - Jumlah kapsul vitamiin A yang di...  valueCodeableConcept   
101                 MedicationRequest - Peresepan Obat                   NaN   
102                MedicationDispense - Pemberian obat                   NaN   
103       Observation - Pemeriksaan Layanan Gizi Buruk  valueCodeableConcept   
104                     Procedure - Intervensi Lainnya                   NaN   

          Resource_Type                             Payload_Name  \
0           Observation                        Berat Badan Lahir   
1           Observation                              Berat Badan   
2           Observation             Kenaikan Berat Badan Adekuat   
3           Observation         Persentase Penurunan Berat Badan   
4           Observation                                     BB/U   
..                  ...                                      ...   
100         Observation  Jumlah kapsul vitamiin A yang diberikan   
101   MedicationRequest                           Peresepan Obat   
102  MedicationDispense                           Pemberian obat   
103         Observation           Pemeriksaan Layanan Gizi Buruk   
104           Procedure                       Intervensi Lainnya   

    QuestionnaireResponse_Link QuestionnaireResponse_LinkId Text  ...  \
0                          NaN                          NaN  NaN  ...   
1                          NaN                          NaN  NaN  ...   
2                          NaN                          NaN  NaN  ...   
3                          NaN                          NaN  NaN  ...   
4                          NaN                          NaN  NaN  ...   
..                         ...                          ...  ...  ...   
100                        NaN                          NaN  NaN  ...   
101                        NaN                          NaN  NaN  ...   
102                        NaN                          NaN  NaN  ...   
103                        NaN                          NaN  NaN  ...   
104                        NaN                          NaN  NaN  ...   

    valueQuantity_unit valueQuantity_System         valueQuantity_code  \
0                    g                    g  http://unitsofmeasure.org   
1                   kg                   kg  http://unitsof

In [6]:
# title: for constructing QuestionnaireResponse

# defining df_qr where df.column(Resource_Type= "QuestionnaireResponse")
df_qr= df_main[df_main["Resource_Type"] == "QuestionnaireResponse"].copy()

# Calculate cardinality and create a new column
df_qr['Cardinality'] = df_qr['QuestionnaireResponse_LinkId'].apply(lambda x: len(x.split('.')))

# Now df_qr contains a new column 'Cardinality' with the count of integers in each LinkId
df_qr= df_qr.sort_values(by= ["Cardinality", "QuestionnaireResponse_LinkId"], ascending= [False, True])
df_qr

Sequence                 Main_Folder  \
21       21                      SiGIZI   
22       21                      SiGIZI   
23       21                      SiGIZI   
24       21                      SiGIZI   
25       21                      SiGIZI   
27       22                      SiGIZI   
28       22                      SiGIZI   
30       23                      SiGIZI   
31       23                      SiGIZI   
32       23                      SiGIZI   
33       23                      SiGIZI   
78       67  Gizi Buruk (Pelita Kesmas)   
79       67  Gizi Buruk (Pelita Kesmas)   
80       67  Gizi Buruk (Pelita Kesmas)   
94       80  Gizi Buruk (Pelita Kesmas)   
95       81  Gizi Buruk (Pelita Kesmas)   
20       21                      SiGIZI   
26       22                      SiGIZI   
29       23                      SiGIZI   
77       67  Gizi Buruk (Pelita Kesmas)   
93       80  Gizi Buruk (Pelita Kesmas)   

                              Folder_Name  \
21                          Skrining Gizi   
22                          Skrining Gizi   
23                          Skrining Gizi   
24                          Skrining Gizi   
25                          Skrining Gizi   
27                          Skrining Gizi   
28                          Skrining Gizi   
30                          Skrining Gizi   
31                          Skrining Gizi   
32                          Skrining Gizi   
33                          Skrining Gizi   
78                      Faktor Determinan   
79                      Faktor Determinan   
80                      Faktor Determinan   
94  Tambahan pemberian MPASI (Monitoring)   
95  Tambahan pemberian MPASI (Monitoring)   
20                          Skrining Gizi   
26                          Skrining Gizi   
29                          Skrining Gizi   
77                      Faktor Determinan   
93  Tambahan pemberian MPASI (Monitoring)   

                                           JSON_Title            Value_Type  \
21  QuestionnaireResponse - Risiko Malnutrisi - An...          valueBoolean   
22  QuestionnaireResponse - Risiko Malnutrisi - An...          valueBoolean   
23  QuestionnaireResponse - Risiko Malnutrisi - An...          valueBoolean   
24  QuestionnaireResponse - Risiko Malnutrisi - An...          valueBoolean   
25  QuestionnaireResponse - Risiko Malnutrisi - An...          valueBoolean   
27  QuestionnaireResponse - Risiko Malnutrisi - De...  valueCodeableConcept   
28  QuestionnaireResponse - Risiko Malnutrisi - De...          valueBoolean   
30  QuestionnaireResponse - Risiko Malnutrisi - Ib...          valueBoolean   
31  QuestionnaireResponse - Risiko Malnutrisi - Ib...          valueBoolean   
32  QuestionnaireResponse - Risiko Malnutrisi - Ib...          valueBoolean   
33  QuestionnaireResponse - Risiko Malnutrisi - Ib...          valueBoolean   
78  QuestionnaireResponse - Faktor Determinan Gizi...  valueCodeableConcept   
79  QuestionnaireResponse - Faktor Determinan Gizi...  valueCodeableConcept   
80  QuestionnaireResponse - Faktor Determinan Gizi...  valueCodeableConcept   
94  QuestionnaireResponse - Monitoring MP-ASI Gizi...  valueCodeableConcept   
95  QuestionnaireResponse - Monitoring MP-ASI Gizi...          valueBoolean   
20  QuestionnaireResponse - Risiko Malnutrisi - An...                   NaN   
26  QuestionnaireResponse - Risiko Malnutrisi - De...                   NaN   
29  QuestionnaireResponse - Risiko Malnutrisi - Ib...                   NaN   
77  QuestionnaireResponse - Faktor Determinan Gizi...                   NaN   
93  QuestionnaireResponse - Monitoring MP-ASI Gizi...                   NaN   

            Resource_Type                          Payload_Name  \
21  QuestionnaireResponse   Risiko Malnutrisi - Anak ≤ 18 tahun   
22  QuestionnaireResponse   Risiko Malnutrisi - Anak ≤ 18 tahun   
23  QuestionnaireResponse   Risiko Malnutrisi - Anak ≤ 18 tahun   
24  QuestionnaireResponse   Risiko Malnutrisi - Anak ≤ 18 

In [7]:
# Initialize an empty dictionary to hold items by their parent_linkId
items_by_link_id = {}

def find_or_create_item(link_id_parts, items_dict):
    # Determine parent_linkId and current linkId
    parent_linkId = '.'.join(link_id_parts[:-1])
    link_id = '.'.join(link_id_parts)

    # Create or find the current item
    if link_id not in items_dict:
        items_dict[link_id] = QuestionnaireResponseItem.construct(linkId=link_id)

    # If there's a parent linkId, recursively ensure parent items exist
    if parent_linkId and parent_linkId not in items_dict:
        find_or_create_item(link_id_parts[:-1], items_dict)

    # Append the current item to its parent's 'item' list if it has a parent
    if parent_linkId:
        parent_item = items_dict[parent_linkId]
        if not hasattr(parent_item, 'item') or parent_item.item is None:
            parent_item.item = []
        if items_dict[link_id] not in parent_item.item:
            parent_item.item.append(items_dict[link_id])

    return items_dict[link_id]

for index, row in df_qr.iterrows():
    link_id_parts = row['QuestionnaireResponse_LinkId'].split('.')
    
    # Create or find the current item
    current_item = find_or_create_item(link_id_parts, items_by_link_id)
    current_item.text = row['Text']
    
    # Create and add answer
    if row["Value_Type"] == "valueString":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueString = row['valueString']
        current_item.answer = [answer]
    elif row["Value_Type"] == "valueInteger":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueInteger= row ['valueInteger']
        current_item.answer = [answer]
    elif row["Value_Type"] == "valueQuantity":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueQuantity = Quantity(
            value= row["valueQuantity_value"],
            system= row["valueQuantity_System"],
            unit= row["valueQuantity_unit"],
            code= row["valueQuantity_code"]
        )
        current_item.answer = [answer]
    elif row["Value_Type"]== "valueBoolean":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueBoolean= row["valueBoolean"]
        current_item.answer = [answer]
    elif row["Value_Type"]== "valueDateTime":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueDateTime= row["valueDateTime"] + "00:00"
        current_item.answer = [answer]
    elif row["Value_Type"]== "valueCodeableConcept":
        answer = QuestionnaireResponseItemAnswer.construct()
        answer.valueCoding = Coding(
            system= row["CodeableConcept_System"],
            code= row["CodeableConcept_Code"],
            display= row["CodeableConcept_Display"]
        )
        current_item.answer = [answer]
    else: None

import json
# Filter out top-level items to add to the QuestionnaireResponse
top_level_items = [item for link_id, item in items_by_link_id.items() if '.' not in link_id]

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(items_by_link_id)



{   '1': QuestionnaireResponseItem(resource_type='QuestionnaireResponseItem', fhir_comments=None, extension=None, id=None, modifierExtension=None, answer=None, definition=None, definition__ext=None, item=[QuestionnaireResponseItem(resource_type='QuestionnaireResponseItem', fhir_comments=None, extension=None, id=None, modifierExtension=None, answer=[QuestionnaireResponseItemAnswer(resource_type='QuestionnaireResponseItemAnswer', fhir_comments=None, extension=None, id=None, modifierExtension=None, item=None, valueAttachment=None, valueBoolean=True, valueBoolean__ext=None, valueCoding=None, valueDate=None, valueDate__ext=None, valueDateTime=None, valueDateTime__ext=None, valueDecimal=None, valueDecimal__ext=None, valueInteger=None, valueInteger__ext=None, valueQuantity=None, valueReference=None, valueString=None, valueString__ext=None, valueTime=None, valueTime__ext=None, valueUri=None, valueUri__ext=None)], definition=None, definition__ext=None, item=None, linkId='1.1', linkId__ext=None,

In [9]:
df_test= df_main.copy()
df_test= df_test[df_test["Resource_Type"] == "Observation"]
df_test= df_test[df_test["Sequence"].astype("Int64") < 3]
row= df_test.loc[0]

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type : "string"}
Org_Id = "{{Org_id}}" # @param {type : "string"}
MR_Gen= random_with_N_digits(4)
Observation_MRID= '202403A000' + str(MR_Gen)
identifier_value= f"{Observation_MRID}"
Patient_Id = "{{Patient_Id}}" # @param {type : "string"}
Patient_PMID= "{{Patient_Id}}" # @param {type : "string"}
Patient_String = "{{Patient_Name}}" # @param {type : "string"}
Practitioner_Id = "{{Practitioner_Id}}" # @param {type : "string"}
Practitioner_String = "{{Practitioner_Name}}" # @param {type : "string"}
Location_Id ="{{Location_id}}" # @param {type : "string"}
Location_String = "{{Location_name}}" # @param {type : "string"}
QuestionnaireResponse_Answerer= "Parents" # @param ["Parents", "Self"] {type: "string"}

DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

x = 1

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = 'Patient/{Patient_Id}'
Subject = Subject.format(Patient_Id=Patient_Id)
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

identifier_system= b'http://sys-ids.kemkes.go.id/Observation/{{Org_id}}'
identifier_value= '202403A000' + str(MR_Gen)
Main_Folder = row['Main_Folder']
Folder_Name = row['Folder_Name']
JSON_Name = row['JSON_Title']

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

Resource_Type= row["Resource_Type"]
Observation_Status= row["Status"]
Observation_Category= row['Category']
Observation_CodeCodingSystem = row['Code_System']
Observation_CodeCodingCode = row['Code_Code']
Observation_CodeCodingDisplay = row['Code_Display']
Observation_valueType = row['Value_Type']
Observation_valueInteger = row['valueInteger']
Observation_valueCodeSystem = row['CodeableConcept_System']
Observation_valueCodeCode = row['CodeableConcept_Code']
Observation_valueCodeDisplay = row['CodeableConcept_Display']
Observation_valueBoolean = row['valueBoolean']
Observation_valueQuantityValue = row['valueQuantity_value']
Observation_valueQuantityUnit = row['valueQuantity_unit']
Observation_valueQuantitySystem = row['valueQuantity_System']
Observation_valueQuantityCode = row['valueQuantity_code']
Observation_valueString = row['valueString']
#Observation_valueDateTime= row["valueDateTime"]
ObservationInterpretation_Type= row['Interpretation_Used']
ObservationInterpretation_System= row['Interpretation_System']
ObservationInterpretation_Code= row['Interpretation_Code']
ObservationInterpretation_Display= row ['Interpretation_Display']
DateTime= DateTime + timedelta(seconds= 60)

# Set up category
category_coding = Coding(
    system=ObservationCategory_Dict[Observation_Category]["system"], 
    code=ObservationCategory_Dict[Observation_Category]["code"], 
    display=ObservationCategory_Dict[Observation_Category]["code"]
    )

# Set up code
observation_coding = Coding(
    system=Observation_CodeCodingSystem, 
    code=Observation_CodeCodingCode, 
    display=Observation_CodeCodingDisplay
    )

my_com= ObservationComponent.construct()
my_com.code= CodeableConcept(coding= [observation_coding])
my_com.valueQuantity= Quantity(value= Observation_valueQuantityValue, code= Observation_valueQuantityCode, unit= Observation_valueQuantityUnit, system= Observation_valueQuantitySystem)

my_com= ObservationComponent_IDFHIR()
my_com.set_value(Observation_valueType)
my_com= my_com.component

my_obs= Observation_IDFHIR()
my_obs.observation.component= [my_com]
my_obs.set_value(Observation_valueType)

my_obs= my_obs.to_json()

print(my_obs)


{
    "resourceType": "Observation",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/Observation/{{Org_id}}",
            "value": "202403A0007588"
        }
    ],
    "status": "final",
    "category": [
        {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                    "code": "survey",
                    "display": "survey"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "8339-4",
                "display": "Birth weight Measured"
            }
        ]
    },
    "subject": {
        "reference": "Patient/{{Patient_Id}}",
        "display": "{{Patient_Name}}"
    },
    "encounter": {
        "reference": "Encounter/{{Encounter_uuid}}",
        "display": "Kunjungan {{Patient_Name}} pada tanggal 02 Sep 2023"
    },
    "effectiveDateT

In [72]:
# redefining df to drop duplicated sequence
df = df_main.drop_duplicates(subset='Sequence', keep='first')

Sequence                 Main_Folder  \
0          1                      SiGIZI   
1          2                      SiGIZI   
2          3                      SiGIZI   
3          4                      SiGIZI   
4          5                      SiGIZI   
..       ...                         ...   
100       86  Gizi Buruk (Pelita Kesmas)   
101       87  Gizi Buruk (Pelita Kesmas)   
102       88  Gizi Buruk (Pelita Kesmas)   
103       89  Gizi Buruk (Pelita Kesmas)   
104       90  Gizi Buruk (Pelita Kesmas)   

                                    Folder_Name  \
0                                  Antropometri   
1                                  Antropometri   
2                                  Antropometri   
3                                  Antropometri   
4                                  Antropometri   
..                                          ...   
100  Penanganan yang diberikan untuk rawat inap   
101  Penanganan yang diberikan untuk rawat inap   
102  Penanganan yang diberikan untuk rawat inap   
103  Penanganan yang diberikan untuk rawat inap   
104  Penanganan yang diberikan untuk rawat inap   

                                            JSON_Title            Value_Type  \
0                      Observation - Berat Badan Lahir         valueQuantity   
1                            Observation - Berat Badan         valueQuantity   
2           Observation - Kenaikan Berat Badan Adekuat          valueBoolean   
3       Observation - Persentase Penurunan Berat Badan  valueCodeableConcept   
4                                   Observation - BB/U         valueQuantity   
..                                                 ...                   ...   
100  Observation - Jumlah kapsul vitamiin A yang di...  valueCodeableConcept   
101                 MedicationRequest - Peresepan Obat                   NaN   
102                MedicationDispense - Pemberian obat                   NaN   
103       Observation - Pemeriksaan Layanan Gizi Buruk  valueCodeableConcept   
104                     Procedure - Intervensi Lainnya                   NaN   

          Resource_Type                             Payload_Name  \
0           Observation                        Berat Badan Lahir   
1           Observation                              Berat Badan   
2           Observation             Kenaikan Berat Badan Adekuat   
3           Observation         Persentase Penurunan Berat Badan   
4           Observation                                     BB/U   
..                  ...                                      ...   
100         Observation  Jumlah kapsul vitamiin A yang diberikan   
101   MedicationRequest                           Peresepan Obat   
102  MedicationDispense                           Pemberian obat   
103         Observation           Pemeriksaan Layanan Gizi Buruk   
104           Procedure                       Intervensi Lainnya   

    QuestionnaireResponse_Link QuestionnaireResponse_LinkId Text  ...  \
0                          NaN                          NaN  NaN  ...   
1                          NaN                          NaN  NaN  ...   
2                          NaN                          NaN  NaN  ...   
3                          NaN                          NaN  NaN  ...   
4                          NaN                          NaN  NaN  ...   
..                         ...                          ...  ...  ...   
100                        NaN                          NaN  NaN  ...   
101                        NaN                          NaN  NaN  ...   
102                        NaN                          NaN  NaN  ...   
103                        NaN                          NaN  NaN  ...   
104                        NaN                          NaN  NaN  ...   

    valueQuantity_unit valueQuantity_System         valueQuantity_code  \
0                    g                    g  http://unitsofmeasure.org   
1                   kg                   kg  http://unitsof

In [37]:
# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type : "string"}
Org_Id = "{{Org_id}}" # @param {type : "string"}
MR_Gen= random_with_N_digits(4)
Observation_MRID= '202403A000' + str(MR_Gen)
identifier_value= f"{Observation_MRID}"
Patient_Id = "{{Patient_Id}}" # @param {type : "string"}
Patient_PMID= "{{Patient_Id}}" # @param {type : "string"}
Patient_String = "{{Patient_Name}}" # @param {type : "string"}
Practitioner_Id = "{{Practitioner_Id}}" # @param {type : "string"}
Practitioner_String = "{{Practitioner_Name}}" # @param {type : "string"}
Location_Id ="{{Location_id}}" # @param {type : "string"}
Location_String = "{{Location_name}}" # @param {type : "string"}
QuestionnaireResponse_Answerer= "Parents" # @param ["Parents", "Self"] {type: "string"}

DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

x = 1

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = 'Patient/{Patient_Id}'
Subject = Subject.format(Patient_Id=Patient_Id)
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

if QuestionnaireResponse_Answerer== "Self":
  QuestionnaireResponse_Source= Subject
elif QuestionnaireResponse_Answerer== "Parents":
  QuestionnaireResponse_Source= Reference(reference= b"RelatedPerson/{{RelatedPerson_uuid}}", display= "Orang Tua Pasien")

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#Operation based on observation input choices
for index, row in df.iterrows():
  identifier_system= b'http://sys-ids.kemkes.go.id/Observation/{{Org_id}}'
  identifier_value= '202403A000' + str(MR_Gen)
  Main_Folder = row['Main_Folder']
  Folder_Name = row['Folder_Name']
  JSON_Name = row['JSON_Title']
  Resource_Type= row["Resource_Type"]
  if Resource_Type == "Observation":
    Observation_Status= row["Status"]
    Observation_Category= row['Category']
    Observation_CodeCodingSystem = row['Code_System']
    Observation_CodeCodingCode = row['Code_Code']
    Observation_CodeCodingDisplay = row['Code_Display']
    Observation_valueType = row['Value_Type']
    Observation_valueInteger = row['valueInteger']
    Observation_valueCodeSystem = row['CodeableConcept_System']
    Observation_valueCodeCode = row['CodeableConcept_Code']
    Observation_valueCodeDisplay = row['CodeableConcept_Display']
    Observation_valueBoolean = row['valueBoolean']
    Observation_valueQuantityValue = row['valueQuantity_value']
    Observation_valueQuantityUnit = row['valueQuantity_unit']
    Observation_valueQuantitySystem = row['valueQuantity_System']
    Observation_valueQuantityCode = row['valueQuantity_code']
    Observation_valueString = row['valueString']
    ObservationInterpretation_Type= row['Interpretation_Used']
    ObservationInterpretation_System= row['Interpretation_System']
    ObservationInterpretation_Code= row['Interpretation_Code']
    ObservationInterpretation_Display= row ['Interpretation_Display']
    DateTime= DateTime + timedelta(seconds= 60)

    # Set up category
    category_coding = Coding(
        system=ObservationCategory_Dict[Observation_Category]["system"], 
        code=ObservationCategory_Dict[Observation_Category]["code"], 
        display=ObservationCategory_Dict[Observation_Category]["code"]
        )
    
    # Set up code
    observation_coding = Coding(
        system=Observation_CodeCodingSystem, 
        code=Observation_CodeCodingCode, 
        display=Observation_CodeCodingDisplay
        )
    
    resource= Observation_IDFHIR()
    resource.set_value(Observation_valueType)
    resource.set_interpretation(ObservationInterpretation_Type)
    resource= resource.to_json()
  
  elif Resource_Type== "Procedure":
    Procedure_Status= row["Status"]
    Procedure_CategorySystem= row["Category_System"]
    Procedure_CategoryCode= row["Category_Code"]
    Procedure_CategoryDisplay= row["Category_Display"]
    Procedure_CodeCodingSystem= row["Code_System"]
    Procedure_CodeCodingCode= row["Code_Code"]
    Procedure_CodeCodingDisplay= row["Code_Display"]
    Procedure_Performer= ProcedurePerformer(actor= Performer)
    # Set up category
    category_coding = Coding(
        system= Procedure_CategorySystem, 
        code=Procedure_CategoryCode, 
        display=Procedure_CategoryDisplay
        )
    # Set up code
    Procedure_coding = Coding(
        system=Procedure_CodeCodingSystem, 
        code=Procedure_CodeCodingCode, 
        display=Procedure_CodeCodingDisplay
        )
    DateTime= DateTime + timedelta(minutes= 15)
    resource= Procedure_IDFHIR()
    resource= resource.to_json()

  elif Resource_Type== "Condition":
    Condition_Status = row["Status"]
    Condition_Category= row["Category"]
    Condition_CodeCodingSystem = row["Code_System"]
    Condition_CodeCodingCode = row["Code_Code"]
    Condition_CodeCodingDisplay = row["Code_Display"]
    ClinicalStatus_Coding= Coding(
            system= ConditionClinicalStatus_Dict[Condition_Status]["system"],
            code= ConditionClinicalStatus_Dict[Condition_Status]["code"],
            display= ConditionClinicalStatus_Dict[Condition_Status]["display"]
        )
    Condition_ClinicalStatus= CodeableConcept(
        coding= [ClinicalStatus_Coding]
    )
    # Set up category
    category_coding = Coding(
        system= ConditionCategory_Dict[Condition_Category]["system"], 
        code=ConditionCategory_Dict[Condition_Category]["code"], 
        display=ConditionCategory_Dict[Condition_Category]["display"]
        )
    # Set up code
    Condition_coding = Coding(
        system=Condition_CodeCodingSystem, 
        code=Condition_CodeCodingCode, 
        display=Condition_CodeCodingDisplay
        )
    DateTime= DateTime + timedelta(seconds= 60)
    resource= Condition_IDFHIR()
    resource= resource.to_json()

  elif Resource_Type== "AllergyIntolerance":
    AllergyIntolerance_ClinicalStatus = row["Status"]
    AllergyIntolerance_Category= row["Category"]
    AllergyIntolerance_CodeCodingSystem = row["Code_System"]
    AllergyIntolerance_CodeCodingCode = row["Code_Code"]
    AllergyIntolerance_CodeCodingDisplay = row["Code_Display"]
    #set up ClinicalStatus
    AllergyIntolerance_ClinicalStatus_Coding= Coding(
        system= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["system"],
        code= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["code"],
        display= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["display"],
    )
    # Set up code
    AllergyIntolerance_coding = Coding(
        system=AllergyIntolerance_CodeCodingSystem, 
        code=AllergyIntolerance_CodeCodingCode, 
        display=AllergyIntolerance_CodeCodingDisplay
        )
    DateTime= DateTime + timedelta(seconds= 60)
    resource= AllergyIntolerance_IDFHIR()
    resource= resource.to_json()

  elif Resource_Type== "QuestionnaireResponse":
    DateTime= DateTime + timedelta(minutes= 5)
    qr= QuestionnaireResponse.construct(
      identifier= Identifier(system= b'http://sys-ids.kemkes.go.id/QuestionnaireResponse/{{Org_id}}', value= identifier_value),
      status= row["Status"],
      questionnaire= row["QuestionnaireResponse_Link"],
      subject= Subject,
      encounter= Encounter,
      authored= DateTime,
      source= QuestionnaireResponse_Source,
      author= Performer,
      item= items_by_link_id[row["QuestionnaireResponse_LinkId"]]
    )
    resource= qr
    resource= resource.json(indent= 4)

  else: resource= ""

  print(row["Sequence"]," - ",Main_Folder,'\n',Folder_Name,'\n',JSON_Name)
  num_rows = resource.count('\n')
  html_text = f'''
  <textarea id="clipboard-text" rows="{num_rows}" cols="150">{resource}</textarea>
  <button onclick="copyToClipboard()">Copy text</button>
  <script>
  function copyToClipboard() {{
    var copyText = document.getElementById("clipboard-text");
    copyText.select();
    navigator.clipboard.writeText(copyText.value);
    }}
  </script>
  '''
  display(HTML(html_text))
  print('=======================================================','\n','\n','\n','\n')

  #Add indentation to every line in Result_JSON
  Result_JSON = '\n'.join('\t'+ line for line in resource.split('\n'))
  formatted_json = BRU_metadata.format(
     Resource_Type= Resource_Type,
     Filename= JSON_Name,
     Sequence= row['Sequence'],
     Result_JSON= Result_JSON
  )
  
  # Define the target folder and file name
  folder_path = f'BRU Results\{Main_Folder}\{Folder_Name}'  # Update this path
  file_string= JSON_Name.replace("/", " per ")
  file_string= file_string.replace("\n", "")
  file_string= file_string.replace(">", "lebih dari ")
  file_name = f'{file_string}.bru'

  # Ensure the folder exists
  os.makedirs(folder_path, exist_ok=True)
  
  #Joining paths as full path and write the JSON inside that path
  full_path = os.path.join(folder_path, file_name)
  open(full_path, 'w', encoding= "utf-8").write(formatted_json)
  
  #Adding along sequence numbers for reuse
  x += 1
  MR_Gen += 1


NameError: name 'df' is not defined